In [1]:
# nuclio: ignore
import nuclio

In [2]:
# Copyright 2018 Iguazio
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import numpy as np
import pandas as pd
from mlrun.execution import MLClientCtx


def load_dataset(
        context: MLClientCtx,
        dataset: str,
        name: str = '',
        file_ext: str = 'parquet',
        params: dict = {}
) -> None:
    """Loads a scikit-learn toy dataset for classification or regression

    The following datasets are available ('name' : desription):

        'boston'          : boston house-prices dataset (regression)
        'iris'            : iris dataset (classification)
        'diabetes'        : diabetes dataset (regression)
        'digits'          : digits dataset (classification)
        'linnerud'        : linnerud dataset (multivariate regression)
        'wine'            : wine dataset (classification)
        'breast_cancer'   : breast cancer wisconsin dataset (classification)

    The scikit-learn functions return a data bunch including the following items:
    - data              the features matrix
    - target            the ground truth labels
    - DESCR             a description of the dataset
    - feature_names     header for data

    The features (and their names) are stored with the target labels in a DataFrame.

    For further details see https://scikit-learn.org/stable/datasets/index.html#toy-datasets

    :param context:    function execution context
    :param dataset:    name of the dataset to load
    :param name:       artifact name (defaults to dataset)
    :param file_ext:   output file_ext: parquet or csv
    :param params:     params of the sklearn load_data method
    """
    dataset = str(dataset)
    # reach into module and import the appropriate load_xxx function
    pkg_module = 'sklearn.datasets'
    fname = f'load_{dataset}'

    pkg_module = __import__(pkg_module, fromlist=[fname])
    load_data_fn = getattr(pkg_module, fname)

    data = load_data_fn(**params)
    feature_names = data['feature_names']

    # create the toy dataset
    xy = np.concatenate([data['data'], data['target'].reshape(-1, 1)], axis=1)
    if hasattr(feature_names, 'append'):
        # its a list
        feature_names.append('labels')
    else:
        # its an array
        feature_names = np.append(feature_names, 'labels')
    df = pd.DataFrame(data=xy, columns=feature_names)

    # log and upload the dataset
    context.log_dataset(name or dataset, df=df, format=file_ext, index=False)

In [3]:
# nuclio: end-code

### mlconfig

In [4]:
from mlrun import mlconf

In [5]:
mlconf.dbpath = mlconf.dbpath or './'
mlconf.dbpath

'http://mlrun-api:8080'

In [6]:
vcs_branch = 'development'
base_vcs = f'https://raw.githubusercontent.com/mlrun/functions/{vcs_branch}/'

mlconf.hub_url = mlconf.hub_url or base_vcs + f'{name}/function.yaml'
mlconf.hub_url

'/User/repos/functions/{name}/function.yaml'

In [7]:
import os
mlconf.artifact_path = mlconf.artifact_path or f'{os.environ["V3IO_HOME"]}/artifacts'
mlconf.artifact_path

'/User/artifacts'

In [8]:
import os
TAG = os.environ['MLRUN_COMMIT']

### save

In [9]:
from mlrun import code_to_function 
# create job function object from notebook code
fn = code_to_function("load_dataset", kind="job", with_doc=True,
                      handler=load_dataset, 
                      image=f"mlrun/ml-models:{TAG}")

# add metadata (for templates and reuse)
fn.spec.default_handler = "load_dataset"
fn.spec.description = "load a toy dataset from scikit-learn"
fn.metadata.categories = ["datagen", "filesutils"]
fn.metadata.labels = {"author": "yjb"}

fn.save()
fn.export("function.yaml")

[mlrun] 2020-04-27 23:52:04,007 saving function: load-dataset, tag: latest
[mlrun] 2020-04-27 23:52:04,056 function spec saved to path: function.yaml


## tests

In [10]:
from mlrun import import_function

func = import_function("hub://load_dataset")

if "V3IO_HOME" in list(os.environ):
    from mlrun import mount_v3io
    func.apply(mount_v3io())
else:
    # is you set up mlrun using the instructions at https://github.com/mlrun/mlrun/blob/master/hack/local/README.md
    from mlrun.platforms import mount_pvc
    func.apply(mount_pvc('nfsvol', 'nfsvol', '/home/joyan/data'))

In [11]:
from mlrun import NewTask    

task_params = {
    "name"   : "tasks load toy dataset", 
    "params" : {"dataset"     : "wine"}}

### run remotely

In [12]:
run = func.run(NewTask(**task_params), artifact_path=mlconf.artifact_path)

[mlrun] 2020-04-27 23:52:04,106 starting run tasks load toy dataset uid=8ef6292d9dde4c3393ca78451c6b8f57  -> http://mlrun-api:8080
[mlrun] 2020-04-27 23:52:04,212 Job is running in the background, pod: tasks-load-toy-dataset-9mr64
[mlrun] 2020-04-27 23:52:08,185 log artifact wine at /User/artifacts/wine.parquet, size: 17997, db: Y

[mlrun] 2020-04-27 23:52:08,223 run executed, status=completed
final state: succeeded


to track results use .show() or .logs() or in CLI: 
!mlrun get run 8ef6292d9dde4c3393ca78451c6b8f57  , !mlrun logs 8ef6292d9dde4c3393ca78451c6b8f57 
[mlrun] 2020-04-27 23:52:10,438 run executed, status=completed


### or locally

In [13]:
from mlrun import run_local

In [14]:
for dataset in ["wine", "iris", "breast_cancer"]:
    run_local(handler=load_dataset,
              inputs={"dataset": dataset}, artifact_path=mlconf.artifact_path)

[mlrun] 2020-04-27 23:52:10,459 starting run mlrun-bcf268-load_dataset uid=b1738891fde94f5a9fb8613489f9a928  -> http://mlrun-api:8080
[mlrun] 2020-04-27 23:52:11,068 log artifact wine at /User/artifacts/wine.parquet, size: 18006, db: Y



to track results use .show() or .logs() or in CLI: 
!mlrun get run b1738891fde94f5a9fb8613489f9a928 --project default , !mlrun logs b1738891fde94f5a9fb8613489f9a928 --project default
[mlrun] 2020-04-27 23:52:11,135 run executed, status=completed
[mlrun] 2020-04-27 23:52:11,143 starting run mlrun-430024-load_dataset uid=72b17810aeaa4344a2f569301a12c8d4  -> http://mlrun-api:8080
[mlrun] 2020-04-27 23:52:11,223 log artifact iris at /User/artifacts/iris.parquet, size: 5049, db: Y



to track results use .show() or .logs() or in CLI: 
!mlrun get run 72b17810aeaa4344a2f569301a12c8d4 --project default , !mlrun logs 72b17810aeaa4344a2f569301a12c8d4 --project default
[mlrun] 2020-04-27 23:52:11,267 run executed, status=completed
[mlrun] 2020-04-27 23:52:11,274 starting run mlrun-7347b1-load_dataset uid=5012b6c319ba4f529dc0540ea804cc6c  -> http://mlrun-api:8080
[mlrun] 2020-04-27 23:52:11,530 log artifact breast_cancer at /User/artifacts/breast_cancer.parquet, size: 146754, db: Y



to track results use .show() or .logs() or in CLI: 
!mlrun get run 5012b6c319ba4f529dc0540ea804cc6c --project default , !mlrun logs 5012b6c319ba4f529dc0540ea804cc6c --project default
[mlrun] 2020-04-27 23:52:11,643 run executed, status=completed
